In [1]:
import numpy as np , pandas as pd
import pyodbc
import statistics
from tqdm import tqdm
import time
import math
import datetime
from datetime import datetime
from configparser import ConfigParser
import pyodbc


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# # Changed Code for audit integration

# conn.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
# (COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_REPORT',GETDATE(),'Error','RISK_REPORT','RISK_TRANSACTION_SUMMARY_STG',(select max(update_date) from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG))""")

# job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN",conn)
# job_id = job_id.iloc[0,0]
# conn.commit()



## Populating RISK_REPORT_SUMMARY_STG  

In [4]:
import math
def round_func(num):
    if num-math.floor(num)<0.5:
        return math.floor(num)
    return math.ceil(num)

In [5]:
# pyodbc.drivers()
# conn = pyodbc.connect('DRIVER=SQL Server;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# cursor = conn.cursor()
summary_STG_table = pd.read_sql_query("select A.* from Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG A with(nolock) INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT_INCREMENTAL B with(nolock) ON A.REPORT_ID = B.REPORT_ID", conn)




In [6]:
RISK_REPORT_SUMMARY_STG = summary_STG_table[['BATCH_ID','REPORT_ID','YEAR_REPORT','MONTH_REPORT','REPORT_TYPE','CREATE_DATE','UPDATE_DATE']]
RISK_REPORT_SUMMARY_STG

BATCH_ID  REPORT_ID  YEAR_REPORT  MONTH_REPORT REPORT_TYPE  \
0  2022122710111177     113121         2022            12         STR   
1  2022122710111177     113121         2022            12         STR   

  CREATE_DATE UPDATE_DATE  
0  2023-03-04  2023-03-04  
1  2023-03-04  2023-03-04

In [7]:
Tran_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_TRANSACTION with(nolock)", conn)

Tran_risk_table

BATCH_ID  REPORT_ID REPORT_TYPE  RISK_SCORE CREATE_DATE
0    2022102010167354      91013         STR           1  2023-03-02
1    2022110700166687      96813         CTR           0  2023-03-02
2    2022111510138172     100712         STR           1  2023-03-02
3    2022111600196191     101012         CTR           0  2023-03-02
4    2022111810164377     102013         STR           0  2023-03-02
..                ...        ...         ...         ...         ...
232  2023022130361914     126513         NTR           0  2023-03-02
233  2023022130395284     126313         NTR           1  2023-03-02
234  2023022130395284     126314         NTR           1  2023-03-02
235  2023022330338394     127112         NTR           0  2023-03-02
236  2023022500113666     127422         CTR           1  2023-03-02

[237 rows x 5 columns]

### Inserting data in RISK_REPORT_SUMMARY_STG

In [12]:
from tqdm import tqdm

### Creating Table RISK_REPORT Using Tables RISK_TRANSACTION, RISK_PERSON and RISK_GOS

In [13]:
Tran_risk_table = summary_STG_table.merge(Tran_risk_table, how = 'left', on = ['BATCH_ID','REPORT_ID'])

In [14]:
Tran_risk_table = Tran_risk_table.drop(['CREATE_DATE_x','YEAR_REPORT', 'MONTH_REPORT', 'REPORT_TYPE_y','PEER_GROUP_ID', 'OCCUPATION_BUSINESS', 'TRAN_TYP','CR_DR_DEP_WITH',   'TRAN_AMNT', 'CREATE_DATE_y', 'UPDATE_DATE'], axis=1)

In [15]:
Tran_risk_table.rename(columns = {'REPORT_TYPE_x':'REPORT_TYPE'},inplace = True)

In [16]:
Tran_risk_table.drop_duplicates(inplace = True)

In [17]:
Tran_risk_table.rename(columns = {'RISK_SCORE':'TRANSACTION_SCORE'},inplace = True)

In [18]:
Tran_risk_table

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  
0                  1  
1                  1

In [19]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [20]:
Entity_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_PERSON with(nolock)", conn)
conn.close()


In [21]:
Entity_risk_table['ENTITY_TYPE']='I'

In [22]:
Entity_risk_table

PERSON_MASTER_ID        PEER_GROUP_ID  RISK_SCORE  \
0                   1  Government Sector_0          10   
1                   2  Government Sector_0           4   
2                   3  Government Sector_0           4   
3                   4  Government Sector_0           4   
4                   5  Government Sector_0           4   
..                ...                  ...         ...   
351               458     OTHERS_CHANGED_2           3   
352               459             Others_0           1   
353               460             Others_0           1   
354               461             Others_0           1   
355               462             Others_0           1   

                                        RISK_RULE_LIST  \
0                                           P001005000   
1    P002001001_P002001002_P002001004_P002001007_P0...   
2    P002001001_P002001002_P002001004_P002001007_P0...   
3    P002001001_P002001002_P002001004_P002001007_P0...   
4    P002001001_P002001002_P002001004_P002001007_P0...   
..                                                 ...   
351  P002001002_P002001004_P002001005_P002001007_P0...   
352                                         P002001007   
353                                         P002001007   
354                                         P002001007   
355                                         P002001007   

                CREATE_DATE ENTITY_TYPE  
0                       NaT           I  
1   2023-03-02 20:13:22.053           I  
2   2023-03-02 20:13:22.053           I  
3   2023-03-02 20:13:22.057           I  
4   2023-03-02 20:13:22.057           I  
..                      ...         ...  
351 2023-03-02 20:13:22.383           I  
352 2023-03-02 20:13:22.383           I  
353 2023-03-02 20:13:22.383           I  
354 2023-03-02 20:13:22.383           I  
355 2023-03-02 20:13:22.387           I  

[356 rows x 6 columns]

In [23]:
# Tran_risk_table[Tran_risk_table['ENTITY_TYPE']=='O']

In [24]:
merge_TR_ER = Tran_risk_table.merge(Entity_risk_table, how = 'left', left_on= ['ENTITY_ID','ENTITY_TYPE'],right_on = ['PERSON_MASTER_ID','ENTITY_TYPE'])
merge_TR_ER

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  PERSON_MASTER_ID     PEER_GROUP_ID  RISK_SCORE  \
0                  1               372  OTHERS_CHANGED_0           3   
1                  1               377  OTHERS_CHANGED_0           4   

                                      RISK_RULE_LIST             CREATE_DATE  
0  P002001007_P002001008_P002001024_P002001025_P0... 2023-03-02 20:13:22.333  
1  P002001005_P002001007_P002001008_P002001009_P0... 2023-03-02 20:13:22.337

In [25]:
merge_TR_ER.rename(columns = {'RISK_SCORE' :'RISK_SCORE_PERSON'},inplace=True)

In [26]:
merge_TR_ER.drop(columns = ['PERSON_MASTER_ID','PEER_GROUP_ID','CREATE_DATE'],inplace = True)

In [27]:
merge_TR_ER.drop_duplicates(inplace = True)

In [28]:
#  merge_TR_ER = merge_TR_ER[['BATCH_ID','REPORT_ID','REPORT_TYPE','RISK_SCORE','CREATE_DATE']]

In [29]:
 merge_TR_ER

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  RISK_SCORE_PERSON  
0                  1                  3  
1                  1                  4

In [30]:
temp = merge_TR_ER

In [31]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [32]:
Org_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_ORGANIZATION with(nolock)", conn)
conn.close()

In [33]:
Org_risk_table['ENTITY_TYPE']='O'

In [34]:
Org_risk_table

ORGANIZATION_MASTER_ID     PEER_GROUP_ID  RISK_SCORE  \
0                        1  OTHERS_CHANGED_9          10   
1                        2  OTHERS_CHANGED_0           6   
2                        3  OTHERS_CHANGED_0           6   
3                        4  OTHERS_CHANGED_3           3   
4                        5  OTHERS_CHANGED_0           6   
..                     ...               ...         ...   
64                      80  OTHERS_CHANGED_0           5   
65                      81  OTHERS_CHANGED_8           4   
66                      82  OTHERS_CHANGED_9           4   
67                      83  OTHERS_CHANGED_9           4   
68                      84  OTHERS_CHANGED_2           4   

                                       RISK_RULE_LIST             CREATE_DATE  \
0                                          O001006000                     NaT   
1   O002001007_O002001008_O002001012_O002001013_O0... 2023-03-02 21:29:59.237   
2   O002001007_O002001008_O002001012_O002001013_O0... 2023-03-02 21:29:59.240   
3   O002001002_O002001012_O002001013_O002001024_O0... 2023-03-02 21:29:59.240   
4   O002001007_O002001008_O002001012_O002001013_O0... 2023-03-02 21:29:59.240   
..                                                ...                     ...   
64  O002001007_O002001008_O002001012_O002001013_O0... 2023-03-02 21:29:59.287   
65  O002001001_O002001002_O002001004_O002001007_O0... 2023-03-02 21:29:59.287   
66  O002001001_O002001002_O002001004_O002001007_O0... 2023-03-02 21:29:59.287   
67  O002001001_O002001002_O002001004_O002001007_O0... 2023-03-02 21:29:59.287   
68  O002001002_O002001004_O002001005_O002001007_O0... 2023-03-02 21:29:59.290   

   ENTITY_TYPE  
0            O  
1            O  
2            O  
3            O  
4            O  
..         ...  
64           O  
65           O  
66           O  
67           O  
68           O  

[69 rows x 6 columns]

In [35]:
merge_TR_ER = merge_TR_ER.merge(Org_risk_table, how = 'left', left_on= ['ENTITY_ID','ENTITY_TYPE'],right_on = ['ORGANIZATION_MASTER_ID','ENTITY_TYPE'])


In [36]:
merge_TR_ER.drop(columns = ['PEER_GROUP_ID','CREATE_DATE','ORGANIZATION_MASTER_ID'],inplace = True)

In [37]:
merge_TR_ER.rename(columns = {'RISK_SCORE':'ORG_SCORE'},inplace = True)

In [38]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [39]:
GOS_risk_table = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_GOS with(nolock)", conn)

In [40]:
GOS_risk_table.drop(columns = ['CREATE_DATE'],inplace = True)

In [41]:
GOS_risk_table

BATCH_ID  REPORT_ID  RISK_SCORE
0     202206291013154      62415        6.50
1     202206291013154      62416        6.50
2     202206291013154      62417        6.50
3     202206291019813      62412        6.50
4     202206291019813      62413        6.50
..                ...        ...         ...
471  2022122110174526     112613        5.83
472  2022122110174526     112614        5.83
473  2022122110174526     112615        5.83
474  2022122110174526     112616        5.83
475  2022122110174526     112617        5.83

[476 rows x 3 columns]

In [42]:
all_data = merge_TR_ER.merge(GOS_risk_table, how= 'left',left_on = ['BATCH_ID','REPORT_ID'],right_on = ['BATCH_ID','REPORT_ID'])

In [43]:
all_data.rename(columns = {'RISK_SCORE' :'RISK_SCORE_GOS'},inplace=True)

In [44]:
all_data.head()

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  RISK_SCORE_PERSON  ORG_SCORE  RISK_SCORE_GOS  
0                  1                  3        NaN             NaN  
1                  1                  4        NaN             NaN

In [45]:
all_data['RISK_SCORE_GOS'].fillna(0,inplace=True)

In [52]:
all_data['ENTITY_RISK']  = all_data[['RISK_SCORE_PERSON','ORG_SCORE']].bfill(axis=1).iloc[:, 0]

In [53]:
all_data.drop(columns = ['RISK_SCORE_PERSON','ORG_SCORE'],inplace = True)

In [54]:
all_data.drop_duplicates(inplace= True)

In [55]:
all_data['TRANSACTION_SCORE'].fillna(0,inplace =True)
all_data['RISK_SCORE_GOS'].fillna(0,inplace =True)
all_data['ENTITY_RISK'].fillna(0,inplace =True)

In [56]:
all_data

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  RISK_SCORE_GOS  ENTITY_RISK  
0                  1             0.0          3.0  
1                  1             0.0          4.0

In [57]:
all_data['FINAL_SCORE']= (((all_data['TRANSACTION_SCORE']**2)+(all_data['RISK_SCORE_GOS']**2)+(all_data['ENTITY_RISK']**2))/(all_data['TRANSACTION_SCORE']+all_data['RISK_SCORE_GOS']+all_data['ENTITY_RISK']))

In [58]:
all_data['FINAL_SCORE'].fillna(0,inplace  =True)

In [59]:
all_data

BATCH_ID  REPORT_ID REPORT_TYPE  ENTITY_ID ENTITY_TYPE  TRAN_CNT  \
0  2022122710111177     113121         STR        372           I         1   
1  2022122710111177     113121         STR        377           I         1   

   TRANSACTION_SCORE  RISK_SCORE_GOS  ENTITY_RISK  FINAL_SCORE  
0                  1             0.0          3.0          2.5  
1                  1             0.0          4.0          3.4

In [60]:
all_data.rename(columns = {'final_score':'RISK_SCORE'}, inplace = True)

In [61]:
all_data.drop(columns = ['TRANSACTION_SCORE','ENTITY_ID','RISK_SCORE_GOS','ENTITY_RISK','ENTITY_TYPE'],inplace = True)

In [62]:
all_data

BATCH_ID  REPORT_ID REPORT_TYPE  TRAN_CNT  FINAL_SCORE
0  2022122710111177     113121         STR         1          2.5
1  2022122710111177     113121         STR         1          3.4

In [63]:
all_data  = all_data.groupby(['BATCH_ID','REPORT_ID','REPORT_TYPE']).mean()

In [64]:
all_data.reset_index(inplace = True)

In [65]:
all_data.shape

(1, 5)

In [66]:
auto_high = pd.read_sql_query('''select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules '  ''',conn)

In [67]:
auto_high = auto_high['REPORT_ID'].to_list()

In [68]:
all_data = all_data[~all_data['REPORT_ID'].isin(auto_high)]

In [69]:
all_data['FINAL_SCORE'] = all_data['FINAL_SCORE'].astype('float64')

In [70]:
all_data

BATCH_ID  REPORT_ID REPORT_TYPE  TRAN_CNT  FINAL_SCORE
0  2022122710111177     113121         STR         1         2.95

In [71]:
all_data['FINAL_SCORE'] = all_data['FINAL_SCORE'].apply(lambda x: round_func(x)) 

In [72]:
config = ConfigParser()

config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [73]:
for ind,row in all_data.iterrows():
#     print(row['BATCH_ID'])
    
    cursor.execute("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE_WEIGHTED_AVG = {} WHERE REPORT_ID = {} """.format(row['FINAL_SCORE'],row['REPORT_ID'])) 
    conn.commit()
    
conn.close()
    

In [74]:
# all_data[all_data['REPORT_ID']==39201]

In [75]:
config = ConfigParser()

config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [76]:
report_risk = pd.read_sql_query('''select A.* from  FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT A with (nolock) INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_INCREMENTAL B WITH (NOLOCK) ON A.REPORT_ID = B.REPORT_ID ''',conn)

In [77]:
report_risk['RISK_SCORE_WEIGHTED_AVG'].fillna(0,inplace = True)

In [78]:
report_risk

BATCH_ID  REPORT_ID REPORT_TYPE RISK_SCORE CREATE_DATE  \
0  2022122710111177     113121         STR       None  2023-03-03   

   RISK_SCORE_QUERY  RISK_SCORE_WEIGHTED_AVG  
0                 0                        3

In [79]:
report_risk['RISK_SCORE'] = (report_risk['RISK_SCORE_QUERY']**2 + report_risk['RISK_SCORE_WEIGHTED_AVG']**2 )/(report_risk['RISK_SCORE_QUERY'] + report_risk['RISK_SCORE_WEIGHTED_AVG'] )

In [80]:
report_risk['RISK_SCORE'].fillna(0,inplace = True)

In [81]:
report_risk['RISK_SCORE'] = report_risk['RISK_SCORE'].apply(lambda x: round_func(x)) 

In [83]:
auto_high = pd.read_sql_query('''select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules '  ''',conn)

In [84]:
auto_high = auto_high['REPORT_ID'].to_list()

In [85]:
report_risk = report_risk[~report_risk['REPORT_ID'].isin(auto_high)]

In [86]:
for ind,row in report_risk.iterrows():
#     print(row['BATCH_ID'])
#     print("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = {} WHERE REPORT_ID = {} """.format(row['RISK_SCORE'],row['REPORT_ID'])) 

    cursor.execute("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = {} WHERE REPORT_ID = {} """.format(row['RISK_SCORE'],row['REPORT_ID'])) 
    conn.commit()
    
    

UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 3 WHERE REPORT_ID = 113121 


In [87]:
store -r

In [89]:
job_id

384

In [90]:
cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE() , JOB_STATUS = 'Success', no_of_record = {}  where job_run_id = {} '''.format(no_of_records,job_id))
conn.commit()

In [ ]:
conn.close()